In [1]:
# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#import matplotlib.patches as mpatches

# Enable inline plotting
%matplotlib inline

#from IPython.display import display, Latex, clear_output

In [2]:
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import webbpsf, poppy
from pynrc.testing import perform_benchmarks

In [3]:
nrc = pynrc.NIRCam()

[     pynrc:INFO] Initializing SCA 481/A1
[     pynrc:INFO] Updating PSF coeff with fov_pix=33 and oversample=4


In [4]:
# from pynrc.nrc_utils import read_filter, gen_psf_coeff, gen_image_coeff, gen_webbpsf_psf
# from pynrc.nrc_utils import gen_webbpsf_siwfe, field_coeff_resid, field_coeff_func
# from pynrc.nrc_utils import Tel2Sci_info, NIRCam_V2V3_limits, radial_std
# from pynrc.nrc_utils import jl_poly_fit, jl_poly

In [5]:
# # Interpolation and extrapolation
# from scipy.interpolate import griddata, RegularGridInterpolator
# from scipy.ndimage import rotate

In [6]:
# pySIAF stuff for plotting
import pysiaf
from pysiaf.siaf import Siaf
from pysiaf.siaf import plot_main_apertures

siaf = Siaf('NIRCam')
siaf.generate_toc()

In [7]:
from astropy.table import Table

# Benchmarks

In [12]:
kwargs = {
    'fov_pix': 129,
    'oversample': 4,
    'pupil': None,
    'filter': 'F430M',
}

# kwargs = {
#     'fov_pix': 320,
#     'oversample': 2,
#     'pupil': 'CIRCLYOT',
#     'filter': 'F444W',
# }

In [13]:
tdict = perform_benchmarks(do_webbpsf_only=True, **kwargs)

Took 0.68 seconds to init WebbPSF
Took 1.58 seconds to generate WebbPSF PSF


In [14]:
tdict = perform_benchmarks(do_webbpsf=False, **kwargs)

Took 9.55 seconds to generate pyNRC coefficients


[     pynrc:WARNING] Generating WFE Drift coefficients. This may take some time...


WFE Drift: 0 nm
WFE Drift: 1 nm
WFE Drift: 2 nm
WFE Drift: 5 nm
WFE Drift: 10 nm
WFE Drift: 20 nm
WFE Drift: 40 nm
Took 76.04 seconds to generate WFE Drift coefficients


[     pynrc:WARNING] Generating field-dependent coefficients. This may take some time...


V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (2.40, -8.08), (95.6, 1157.3)V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (0.45, -8.39), (1957.9, 865.5)V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (1.60, -9.22), (872.6, 83.1)V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (0.87, -8.78), (1561.1, 501.2)V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (0.88, -7.68), (1558.1, 1541.3)V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (1.28, -7.26), (1176.0, 1941.6)





V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (2.00, -8.79), (488.4, 482.3)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (1.99, -7.68), (494.2, 1542.1)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (1.16, -8.60), (1287.8, 664.0)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (0.44, -9.22), (1958.2, 82.1)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (2.43, -9.21), (89.3, 82.8)
V2/V3 Coordinate

[     pynrc:WARNING] Interpolating coefficient residuals onto regular grid...
[     pynrc:WARNING] Done.


Took 203.89 seconds to generate WFE Field coefficients
Took 0.47 seconds to generate pynrc PSF


In [15]:
tdict = perform_benchmarks(do_webbpsf=False, **kwargs)

Took 1.47 seconds to generate pyNRC coefficients
Took 2.58 seconds to generate WFE Drift coefficients
Took 5.64 seconds to generate WFE Field coefficients
Took 0.47 seconds to generate pynrc PSF
